---

## PRQL

Pronounced `prequel`

prql is written in Rust.

It consists of a curated set of orthogonal transformations, which are combined together to form a pipeline. That makes it easy to compose and extend queries. The language also benefits from modern features, such syntax for dates, ranges and f-strings as well as functions, type checking and better null handling.

For further information, you can visit its [website](https://prql-lang.org/)

You can also practice `prql` in its [Playground](https://prql-lang.org/playground/)

{{< video https://www.youtube.com/embed/IQRRsfavEic?si=V9w19lpa3ge7bKYK width="450" height="325" >}}

## Examples

### Basic select

::: {.panel-tabset}

## SQL 
``` {.sql}
SELECT id
  , first_name
  , age
FROM employees
ORDER BY age
LIMIT 10
```

## PRQL
``` {.sql}
from employees
select {id, first_name, age}
sort age
take 10
```

:::

### Easier core syntax

::: {.panel-tabset}

## SQL 
``` {.sql}
SELECT *
FROM track_plays
WHERE plays > 10000
  AND length BETWEEN 60 AND 240
  AND recorded > DATE '2008-01-01'
  AND released - recorded < INTERVAL 180 DAY
ORDER BY length DESC
```

## PRQL
``` {.sql}
from track_plays
filter plays > 10_000                # Readable numbers
filter (length | in 60..240)         # Ranges with `..`
filter recorded > @2008-01-01        # Simple date literals
filter released - recorded < 180days # Nice interval literals
sort {-length}                       # Concise order direction
```

:::

### Filter

::: {.panel-tabset}

## SQL 
``` {.sql}
SELECT
  country,
  MAX(salary) AS max_salary
FROM
  employees
WHERE
  start_date > DATE '2021-01-01'
GROUP BY
  country
HAVING
  MAX(salary) > 100000
```

## PRQL
``` {.sql}
from employees
# `filter` before aggregations...
filter start_date > @2021-01-01
group country (
  aggregate {max_salary = max salary}
)
# ...and `filter` after aggregations!
filter max_salary > 100_000
```

:::

### Expressions

::: {.panel-tabset}

## SQL 
``` {.sql}
SELECT *,
  started + unfinished AS finished,
  (started + unfinished) / started AS fin_share,
  (started + unfinished) / started / (1 - (started + unfinished) / started)
   AS fin_ratio
FROM
  track_plays
```

## PRQL
``` {.sql}
from track_plays
derive {
  finished = started + unfinished,
  fin_share = finished / started,        # Use previous definitions
  fin_ratio = fin_share / (1-fin_share), # BTW, hanging commas are optional!
}
```

:::

### F-strings

::: {.panel-tabset}

## SQL 
``` {.sql}
SELECT
  CONCAT('https://www.', domain, '.', tld, '/', page) AS url
FROM
  web
```

## PRQL
``` {.sql}
from web
# Just like Python
select url = f"https://www.{domain}.{tld}/{page}"
```

:::

### Windows functions

::: {.panel-tabset}

## SQL 
``` {.sql}
SELECT
  *,
  SUM(paycheck) OVER (
    PARTITION BY employee_id
    ORDER BY
      month ROWS BETWEEN 11 PRECEDING AND CURRENT ROW
  ) AS trail_12_m_comp
FROM
  employees
```

## PRQL
``` {.sql}
from employees
group employee_id (
  sort month
  window rolling:12 (
    derive {trail_12_m_comp = sum paycheck}
  )
)
```

:::

### Customized functions

::: {.panel-tabset}

## SQL 
``` {.sql}
SELECT
  temp_c * 9 / 5 + 32 AS temp_f
FROM
  weather
```

## PRQL
``` {.sql}
let celsius_to_fahrenheit = temp -> temp * 9/5 + 32

from weather
select temp_f = (celsius_to_fahrenheit temp_c)
```

:::

### Top N

::: {.panel-tabset}

## SQL 
``` {.sql}
WITH table_0 AS (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY role
      ORDER BY
        join_date
    ) AS _expr_0
  FROM
    employees
)
SELECT
  *
FROM
  table_0
WHERE
  _expr_0 <= 1
```

## PRQL
``` {.sql}
# Most recent employee in each role
# Quite difficult in SQL...
from employees
group role (
  sort join_date
  take 1
)
```

:::

### Joins

::: {.panel-tabset}

## SQL 
``` {.sql}
SELECT
  employees.employee_id,
  p.role,
  b.vision_coverage
FROM
  employees
  JOIN benefits AS b ON employees.employee_id = b.employee_id
  LEFT JOIN positions AS p ON p.id = employees.employee_id
```

## PRQL
``` {.sql}
from employees
join b=benefits (==employee_id)
join side:left p=positions (p.id==employees.employee_id)
select {employees.employee_id, p.role, b.vision_coverage}
```

:::

### Null handling

::: {.panel-tabset}

## SQL 
``` {.sql}
SELECT
  *,
  COALESCE(channel, 'unknown') AS channel
FROM
  users
WHERE
  last_login IS NOT NULL
  AND deleted_at IS NULL
```

## PRQL
``` {.sql}
from users
filter last_login != null
filter deleted_at == null
derive channel = channel ?? "unknown"
```

:::

### Dialects

::: {.panel-tabset}

## SQL 
``` {.sql}
-- MSSQL
SELECT
  TOP (10) *
FROM
  employees
ORDER BY
  age
```

## PRQL
``` {.sql}
prql target:sql.mssql  # Will generate TOP rather than LIMIT

from employees
sort age
take 10
```

:::

## Conclusions

We can see from above examples, prql has an easier syntax for writing queries.   
Nonetheless, the learning curve and transition to writing new sql syntax is steep and it will take its time.

## Contact

<!-- Avatar -->
<img src="../Pictures/profile2.png" alt="me" width="75" height="80">
<!-- Text with color, font, fontsize and specific size -->
<p style="color:#323232; font-family: Helevetica; font-size: 20px;">Jesus L. Monroy<br>Economist | Data Scientist</p>
<!-- Insert url links in logos -->
<!-- style="padding-left:8px" adds spaces before logo link -->
<!-- Telegram -->
<a href="https://t.me/j3suslm" target="_blank" rel="noreferrer"> <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ef/Telegram_X_2019_Logo.svg/2048px-Telegram_X_2019_Logo.png?size=16&color=3b3b3b" alt="telegram" width="30" height="22" style="padding-left:8px"/>
<!-- Twitter -->
<a href="https://www.twitter.com/sqlalchemist" target="_blank" rel="noreferrer"> <img src="https://toppng.com/public/uploads/preview/twitter-x-new-logo-round-icon-png-11692480241tdbz6jparr.webp?size=16&color=3b3b3b" alt="twitter" width="30" height="22" style="padding-left:8px"/>
<!-- Github -->
<a href="https://github.com/SqlAlchemist/My-portfolio" target="_blank" rel="noreferrer"> <img src="https://icongr.am/devicon/github-original.svg?size=16&color=3b3b3b" alt="github" width="30" height="30" style="padding-left:8px"/>
<!-- Linkedin -->
<a href="https://www.linkedin.com/in/j3sus-lmonroy" target="_blank" rel="noreferrer"> <img src="https://icongr.am/simple/linkedin.svg?size=16&color=3b3b3b" alt="linkedin" width="30" height="30" style="padding-left:8px"/>
<!-- Medium -->
<a href="https://medium.com/@jesus_lmonroy" target="_blank" rel="noreferrer"> <img src="https://cdn1.iconfinder.com/data/icons/social-media-and-logos-12/32/Logo_medium-512.png?size=55&color=3b3b3b" alt="medium" width="30" height="33" style="padding-left:8px"/>